In [2]:
import cv2
import numpy as np
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
# Save file paths so we can access the user images
input_path = '/content/drive/MyDrive/Data_Preprocessing/Input'
output_path = '/content/drive/MyDrive/Data_Preprocessing/Output'

In [42]:
img_files = os.listdir(input_path)

In [7]:
!pip install ultralytics

In [8]:
from ultralytics import YOLO
from PIL import Image

In [9]:
model = YOLO("yolov8m.pt")

In [43]:
full_cords_list = []

# Crop the input image into an isolated photo of a
for input_file in img_files:
  input_file_path = os.path.join(input_path, input_file)
  img = Image.open(input_file_path)
  results = model.predict(input_file_path)
  result = results[0]

  cords_list = []
  box_x_dim = img.size[0]
  box_y_dim = img.size[1]
  zoom_out_dim = 25

  for box in result.boxes:
      cords = box.xyxy.tolist()[0]
      cords = [round(x) for x in cords]
      cords = [max(0, cords[0] - zoom_out_dim), max(0, cords[1] - zoom_out_dim), min(box_x_dim, cords[2] + zoom_out_dim), min(box_y_dim, cords[3] + zoom_out_dim)]
      cords_list.append(cords)

  full_cords_list.append(cords_list)



image 1/1 /content/drive/MyDrive/Data_Preprocessing/Input/Copy of frame_1775.jpg: 384x640 1 bird, 1546.6ms
Speed: 4.6ms preprocess, 1546.6ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)


In [44]:
from google.colab.patches import cv2_imshow

# Write new cropped images into output file so it can be animated
i = 0
for image in img_files:
  test_image = cv2.imread(os.path.join(input_path,image))
  cord = full_cords_list[i]
  for item in cord:
    image = test_image[item[1]:item[3], item[0]:item[2]]
  out_filename = f'frame_{i:04d}.jpg'
  out_path = os.path.join(output_path, out_filename)
  cv2.imwrite(out_path, image)
  i += 1

In [45]:
# Create a clean background with no bird in it
input_file_path = os.path.join(input_path, img_files[0])
img = cv2.imread(input_file_path)

# Get border of the box which the bird is in
minX = full_cords_list[0][0][0]
minY = full_cords_list[0][0][1]
maxX = full_cords_list[0][0][2]
maxY = full_cords_list[0][0][3]

# Take the average pixel values of the surrounding pixels to update the missing
# pixel from the box with the bird
for x in range(maxX - minX):
  for y in range(maxY - minY):
    avgColour = np.zeros((3,))
    colourOne = img[minY + y, minX + x - 1]
    colourTwo = img[minY + y - 1, minX + x - 1]
    colourThree = img[minY + y - 1, minX + x]
    avgColour = (colourOne.astype(np.float32) + colourTwo.astype(np.float32) + colourThree.astype(np.float32)) / 3.0

    avgColour[0] = max(avgColour[0], 0)
    avgColour[1] = max(avgColour[1], 0)
    avgColour[2] = max(avgColour[2], 0)

    avgColour[0] = min(avgColour[0], 255)
    avgColour[1] = min(avgColour[1], 255)
    avgColour[2] = min(avgColour[2], 255)
    img[minY + y, minX + x] = avgColour

background = img

out_filename = f'background.jpg'
out_path = os.path.join(output_path, out_filename)
cv2.imwrite(out_path, img)

True

In [ ]:
# Calculate ratio of bird to size of image to determine speed of flying
ratioX = (maxX - minX) / box_x_dim
ratioY = (maxY - minY) / box_y_dim

speedX = (int) (ratioX * 100 / 10)
speedY = (int) (ratioY * 100 / 30)

In [ ]:
# Save animation as seperate frames
animation_path = '/content/drive/MyDrive/Data_Preprocessing/Animation'
animation_frames_path = '/content/drive/MyDrive/Data_Preprocessing/AnimationFrames'
animation_files = os.listdir(animation_path)

for animation_file in animation_files:
  animation_file_path = os.path.join(animation_path, animation_file)
  video = cv2.VideoCapture(animation_file_path)
  frame_count = 0
  while video.isOpened():
      ret, frame = video.read()
      if not ret:
          break

      # Save the frame as an image file
      frame_filename = f'frame_{frame_count:04d}.jpg'
      frame_path = os.path.join(animation_frames_path, frame_filename)
      cv2.imwrite(frame_path, frame)

      frame_count += 1
  video.release()

In [ ]:
# Paste animation frames on top of background to simulate flying
animation_frames = os.listdir(animation_frames_path)
animation = []
x = 0
y = 0
i = 0

for animation_frame in animation_frames:
  input_file_path = os.path.join(animation_frames_path, animation_frame)
  img = cv2.imread(input_file_path)
  img = cv2.resize(img, (maxX - minX, maxY - minY))
  temp = background.copy()
  temp[minY + y : maxY + y, minX + x : maxX + x] = img
  
  if maxX + x + speedX < box_x_dim:
    x += speedX
  if minY - y - speedY < box_y_dim:
    if i % 4 == 0:
      y -= speedY
  i += 1

  animation.append(temp)

out = cv2.VideoWriter('/content/drive/MyDrive/Data_Preprocessing/Output/FinalOutput.mp4', cv2.VideoWriter_fourcc(*'XVID'), 82, (box_x_dim, box_y_dim))

# Save final video to output folder
for i in range(len(animation)):
  out.write(animation[i])
out.release()